# ジオコーディング結果のフィルタリング

このノートブックでは、以下のテクニックを使って、ジオコーディングの結果を絞り込み、フィルタリングする方法をご紹介します。
 - [出力空間参照の指定](#specify-the-output-spatial-reference)
 - [候補の最大数を指定](#specify-the-maximum-number-of-candidates)
 - [範囲内での検索](#search-within-an-extent)
 - [近接検索](#proximity-searches)
 - [カテゴリでフィルタリング](#category-filtering)

<a id="specify-the-output-spatial-reference"></a>
## 出力空間参照の指定

World Geocoding Service はデフォルトでは WGS84の座標（10進度） での位置情報を候補として返しますが、out_sr パラメーターを設定した場合、出力座標に別の空間参照を指定することも可能です。これは、ジオコーディング結果を表示するマップの空間参照が WGS84 ではない地図アプリケーションの場合に必要となります。たとえば、ArcGIS.com のベースマップはWeb メルカトル座標系を使用しており、座標はメートル単位で表示されます。このようなマップでジオコーディングの結果の候補を正しく表示するには、out_sr パラメーターにWeb メルカトル座標系の Well Known ID (WKID) である out_sr=102100 を指定する必要があります。

WKID の値については <a href="https://developers.arcgis.com/rest/services-reference/projected-coordinate-systems.htm">Projected Coordinate Systems</a> や <a href="https://developers.arcgis.com/rest/services-reference/geographic-coordinate-systems.htm">Geographic Coordinate Systems</a> をご参照ください。

### 例：Web メルカトルで出力座標を指定

In [1]:
from arcgis.geocoding import geocode

In [2]:
from arcgis.gis import GIS
gis = GIS()

In [3]:
esrij = geocode("東京都千代田区平河町2-7-1")[0]

In [4]:
esrij['location']

{'x': 139.740651, 'y': 35.679213000000004}

上記は、デフォルトの空間参照 (WKID:4326) です。out_sr パラメーターを用いて出力空間参照を 102100 に設定することで、Webメルカトル座標系 (WKID:102100) の位置を取得することができます。

In [5]:
esrij2 = geocode("東京都千代田区平河町2-7-1", out_sr=102100)[0]

In [6]:
esrij2['location']

{'x': 15555858.112440556, 'y': 4256570.878412549}

<a id="specify-the-maximum-number-of-candidates"></a>
## 候補の最大数を指定
geocode() メソッドの max_locations パラメーターでは、World Geocoding Service の検索によって返される候補の最大数を指定することができます。デフォルトでは、住所検索では最大 20 個、POI 検索では最大 50 個の候補を返すことができます。例として、max_locations=3 と設定した場合、geocode() メソッドは検索が住所やPOIにマッチするかどうかに関わらず、検索の上位3 個の検索結果を返します。max_locations に値が指定されていない場合は、geocode() メソッドはデフォルトの最大値 20 個までを候補として返します。

### 例：POI 検索の候補の最大数を指定する

In [7]:
matches = geocode("Coffee Shop", out_fields="PlaceName,City,Country", max_locations=3)

max_locations=3 を指定しているので、3つの結果が返ってきます。

In [8]:
len(matches)

3

In [9]:
matches

[{'address': 'Coffee Shop',
  'location': {'x': -122.12005999999997, 'y': 37.89071000000007},
  'score': 100,
  'attributes': {'PlaceName': 'Coffee Shop',
   'City': 'Lafayette',
   'Country': 'USA'},
  'extent': {'xmin': -122.12505999999996,
   'ymin': 37.88571000000007,
   'xmax': -122.11505999999997,
   'ymax': 37.89571000000007}},
 {'address': 'Coffee Shop',
  'location': {'x': -122.05836, 'y': 37.97440000000006},
  'score': 100,
  'attributes': {'PlaceName': 'Coffee Shop',
   'City': 'Concord',
   'Country': 'USA'},
  'extent': {'xmin': -122.06335999999999,
   'ymin': 37.96940000000006,
   'xmax': -122.05336,
   'ymax': 37.97940000000006}},
 {'address': 'Coffee Shop',
  'location': {'x': -122.42151002611212, 'y': 37.785869923394394},
  'score': 100,
  'attributes': {'PlaceName': 'Coffee Shop',
   'City': 'San Francisco',
   'Country': 'USA'},
  'extent': {'xmin': -122.42674999999997,
   'ymin': 37.78084000000006,
   'xmax': -122.41674999999998,
   'ymax': 37.79084000000007}}]

<a id="search-within-an-extent"></a>
## 範囲内での検索

geocode() メソッドの search_extent パラメーターを使用して検索結果を空間的にフィルタリングすることができます。モバイルアプリケーションで特に便利な、検索の範囲を特定のエリアに限定したい場合は、検索するためのエクステントを定義することができます。エクステントの外側の候補は返されません。
エクステントの長方形の座標は、シンプルなカンマ区切りの&lt;左下隅&gt;、&lt;右上隅&gt;の文字列として入力することができます。座標はジオコーダーのデフォルトの空間参照（WGS84）でなければなりません。search_extent パラメーターは、サポートされているすべての検索タイプ（住所、POI、行政区域、郵便番号）で使用することができます。


### 例：デフォルトの空間参照で search_extent を使用して POI を検索

次の例では、東京都千代田区にある郵便局を検索してプロットしています。search_extent パラメーターには、千代田区のジオコーディング結果 (result['extent']) から取得し、geocode() メソッドに渡します。

In [10]:
result = geocode("東京都千代田区")[0]
map = gis.map("東京都千代田区")
map

MapView(layout=Layout(height='400px', width='100%'))

In [11]:
# 千代田区にある郵便局を検索してプロット

posts = geocode("Post Office", result['extent'],  max_locations=100)

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

for post in posts:
    map.draw(post['location'], symbol=symbol)

### 例：search_extent を使用して住所を検索

また、エクステント内の道路の住所を検索することもできます。住所検索に search_extent パラメータが定義されている場合、検索から都市と郵便番号を省略しても、有効な一致を見つけることができます。ただし、search_extent で指定した範囲が大きい場合、住所がエクステント内で複数一致する可能性があり、都市、州、郵便番号、またはその他の識別情報を含めて検索を絞り込む必要がある場合があります。さらに、検索に search_extent の範囲外にある都市や郵便番号が含まれている場合は、一致するものは返されません。以下の例では、search_extent を使用した道路の住所を検索しています。

In [12]:
map = gis.map("Redlands, CA")
map

MapView(layout=Layout(height='400px', width='100%'))

In [14]:
redlands = geocode("Redlands, CA")[0]

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

# Redlands の範囲内で "380 New Yort St "を検索
esrihq = geocode("380 New York St", redlands['extent'])[0]
map.draw(esrihq['location'], symbol=symbol)

In [15]:
redlands['extent']

{'xmin': -117.25558999999994,
 'ymin': 33.98238000000007,
 'xmax': -117.10958999999995,
 'ymax': 34.12838000000007}

<a id="proximity-searches"></a>
## 近接検索

<code>
※ 本ノートブックの翻訳時点では、ArcGIS API for Python の API リファレンスの location パラメーターの説明では、 distance パラメーターと併せて用いるものとして説明されていますが、ラッピングしている ArcGIS REST API のドキュメントでは distance パラメーターは既に廃止されており、パラメーターを渡しても認識されない旨が記載されています。
</code>

ジオコーディングの結果は、通常、検索との関連性や相対的な重要度に応じてソートされます。しかし、特にモバイルアプリケーションではユーザーは現在地に最も近い目的の対象物を見つけることを、より重視している場合があります。このため、geocode() メソッドは、指定された点からの距離に基づいて候補の優先順位付けをサポートしています。location パラメーター や distance パラメーターを指定することで、検索するエリアを定義することができます。location パラメーターはエリアの中心点を表し、distance パラメーターは中心からの半径をメートル単位で定義します。ジオコーディング結果の候補のリストで指定したエリアに近い点が上位に表示されるようになります。

### 例：近接検索で郵便局を探す

In [77]:
map = gis.map("東京都千代田区平河町2-7-1")
map

MapView(layout=Layout(height='400px', width='100%'))

In [78]:
esrij = geocode("東京都千代田区平河町2-7-1")[0]
esrij['location']

{'x': 139.740651, 'y': 35.679213000000004}

東京都千代田区平河町2-7-1に近い郵便局を検索しマップに表示します。

In [90]:
posts = geocode("Post Office", location=esrij['location'], distance=1000)

In [91]:
map.draw(posts[0]['location'])

In [93]:
# 一致したアドレスを表示
for post in posts:
    print(post['attributes']['Match_addr'])

町村会館内郵便局
全共連ビル内郵便局
都道府県会館内郵便局
最高裁判所内郵便局
山王グランドビル内郵便局
一ツ木通局
赤坂一ツ木通郵便局
海事ビル内郵便局
国会内郵便局
半蔵門駅前郵便局
駅前局
ホテルニューオータニ内郵便局
山王パークタワー内郵便局
赤坂通郵便局
赤坂通局
千代田一番町郵便局
麹町本通郵便局
麹町本通局
小松ビル内郵便局
第二霞ケ関郵便局


以下は location と distance のパラメータ-を使用せずに、郵便局を検索した場合の結果です。

In [96]:
posts = geocode("Post Office")
for post in posts:
    print(post['attributes']['LongLabel'])

Post Office, Durga Bazar, Cuttack, Odisha, IND
Post Office, Tharangambadi, Nagapattinam, Tamil Nadu, 609313, IND
Post Office, 4195 Anderson Hwy, Powhatan, VA, 23139, USA
Post Office, 4195 Anderson Hwy, Powhatan, VA, 23139, USA
Post Office, 124 E 8th Ave, Cheyenne, WY, 82001, USA
Post Office, 49 St, Leduc, Alberta, T9E, CAN
Post Office, Caldicot Road, Rogiet, Newport, Gwent, Wales, NP26 3, GBR
Post Office, 149 Caerleon Road, Newport, Gwent, Wales, NP19 7, GBR
Post Office, Skinner Street, Newport, Gwent, Wales, NP20 1, GBR
Post Office, 9 Bridge Street, Newport, Gwent, Wales, NP20 4, GBR
Post Office, St Mary Street, Risca, Newport, Gwent, Wales, NP11 6, GBR
Post Office, King Street, Nantyglo, Ebbw Vale, Gwent, Wales, NP23 4, GBR
Post Office, 9 Beaufort Street, Crickhowell, Powys, Wales, NP8 1, GBR
Post Office, Shortbridge Street, Newtown, Powys, Wales, SY16 2, GBR
Post Office, 96 Phoenix Street, Sandycroft, Deeside, Clwyd, Wales, CH5 2, GBR
Post Office, High Street, Connahs Quay, Deeside,

### 例：近接と search_extent の両方を使用した検索

特定のエリア内の候補のみを返し、候補の場所への近さに応じてソートしたい場合は、search_extent パラメーター、location パラメーター、distance パラメーターを一緒に指定することで、検索範囲を定義できます。

In [97]:
address = "Post Office"
location = esrij['location']
distance =  1000
search_extent = esrij['extent']

In [100]:
posts = geocode(address, search_extent, location, distance)
for post in posts:
    print(post['attributes']['Match_addr'])

町村会館内郵便局


<a id="category-filtering"></a>
## カテゴリでフィルタリング

geocode() メソッドは、住所や場所のタイプを表すカテゴリによる検索のフィルタリングをサポートしています。geocode() の category パラメーターを含めることで、 曖昧な検索による予期せぬ場所や住所の型への誤認マッチを回避することができます。

例えば、"Mammoth "と検索した場合、"Mammoth "という名前の場所は世界にたくさんあるので、 "Mammoth "という名前の都市がいくつか出てきます。


### 例 : category パラメーターを指定せずに "Mammoth" を検索

In [38]:
mammoths = geocode("Mammoth")
for mammoth in mammoths:
    print(mammoth['address'])

Mammoth, Arizona
Mammoth, West Virginia
Mammoth, Pennsylvania
Mammoth, California
Mammoth, Missouri
Mammoth, Montana
Mammoth, Wyoming
Mammoth, Utah
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth


geocode() メソッドの category パラメーターを使用して category=Ski Resort のように指定することで、名前に "Mammoth "を含むスキーリゾートだけが返されます。

### 例 : category パラメーターに Ski Resort を指定して "Mammoth" を検索

In [101]:
mammoths = geocode("Mammoth", category="Ski Resort")
for mammoth in mammoths:
    print(mammoth['address'])

Mammoth Mountain


カテゴリの詳細は、<a href="https://developers.arcgis.com/rest/geocode/api-reference/geocoding-category-filtering.htm">Category filtering</a> をご参照ください。